In [13]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [14]:
# === LOAD DATA ===
with open("D:\Study\Ai_Music_Composer\project/data/generator_training_data.pkl", "rb") as f:
    data = pickle.load(f)

X_notes = data["X_notes"]
X_moods = data["X_moods"]
y = data["y"]
note_to_idx = data["note_to_idx"]
idx_to_note = data["idx_to_note"]
mood_classes = data["mood_encoder"]
mappings = {
    'note_to_idx': note_to_idx,
    'idx_to_note': idx_to_note,
    'mood_encoder': mood_classes
}

with open("note_mappings.pkl", "wb") as f:
    pickle.dump(mappings, f)
SEQ_LEN = X_notes.shape[1]
vocab_size = len(note_to_idx)
num_moods = X_moods.shape[1]

In [ ]:
# === BUILD MODEL ===
note_input = Input(shape=(SEQ_LEN,), name="note_input")
mood_input = Input(shape=(num_moods,), name="mood_input")

embedding = Embedding(input_dim=vocab_size, output_dim=128, input_length=SEQ_LEN)(note_input)
lstm_out = LSTM(256)(embedding)

merged = Concatenate()([lstm_out, mood_input])
dense1 = Dense(256, activation='relu')(merged)
output = Dense(vocab_size, activation='softmax')(dense1)

model = Model(inputs=[note_input, mood_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.summary()

In [ ]:
# === TRAIN ===
EPOCHS = 30
BATCH_SIZE = 64

model.fit([X_notes, X_moods], y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

In [ ]:
# === SAVE MODEL ===
model.save("D:\Study\Ai_Music_Composer\project\models\music_generator.h5")
print("✅ Generator model trained and saved.")